In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import datetime as dt
from salishsea_tools import evaltools as et

In [2]:
# read in stored station info (links station names to locations)
dir0 = '/ocean/eolson/MEOPAR/obs/WADE/ptools_data/ecology/'
sta_df = pd.read_pickle(dir0 + 'sta_df.p')

In [3]:
bottle_fn = dir0 + 'raw/ParkerMacCready2019CTDDataFeb2020.xlsx'
sheet_name = '2018-2019NutrientData'
sheet_chl = '2018-2019ChlaLabData'
bot = pd.read_excel(bottle_fn, sheet_name=sheet_name,engine='openpyxl')
chl = pd.read_excel(bottle_fn, sheet_name=sheet_chl, engine='openpyxl')

In [4]:
bot.dropna(how='all',inplace=True) # drop rows where all values are NaN
chl.dropna(how='all',inplace=True)
bot.dropna(subset=['Sampling Depth'],inplace=True) # drop rows with NaN sampling depths because nominal depths do not appear reliable
chl.dropna(subset=['Sampling Depth'],inplace=True)
# drop rows with no useful data:
bot.dropna(how='all',subset=['NH4_Lab','NO2_Lab','NO3_Lab','PO4_Lab','SiOH4_Lab'],inplace=True)
chl.dropna(how='all',subset=['Chla_Lab'],inplace=True)

In [5]:
# drop rows that appear to have misaligned Sampling depths
#Date	Station	Niskin	Nomdepth	Sampling Depth	CTD Cast Rep	Chla_Lab	Chla_QC	
    #2019-02-26	DNA001	12.0	30	1.136	1.0	1.9216	2.0	
chl.drop(chl.loc[(chl.Date==dt.datetime(2019,2,26))&(chl.Station=='DNA001')&\
                 (chl.Nomdepth==30)&(chl['Sampling Depth']==1.9216)].index,inplace=True)
# also remove from chl: 
#       2018-02-01  HCB010     9.0       30          10.545 sample is probably actually 30 m based on nuts
#       2018-03-09  CRR001     9.0       30         101.554 "
chl.drop(chl.loc[(chl.Date==dt.datetime(2018,2,1))&(chl.Station=='HCB010')&\
                 (chl.Nomdepth==30)&(chl['Sampling Depth']==10.545)].index,inplace=True)
chl.drop(chl.loc[(chl.Date==dt.datetime(2018,3,9))&(chl.Station=='CRR001')&\
                 (chl.Nomdepth==30)&(chl['Sampling Depth']==101.554)].index,inplace=True)
#remove from chl and nuts:
#       Date:2019-10-30 Station: ADM001  all 3 values (Nom depth 0,10,30 but sampling depth all ~126.7)
chl.drop(chl.loc[(chl.Date==dt.datetime(2019,10,30))&(chl.Station=='ADM001')].index,inplace=True)
bot.drop(chl.loc[(chl.Date==dt.datetime(2019,10,30))&(chl.Station=='ADM001')].index,inplace=True)

In [6]:
# average over rows having identical  (Date,Station,Niskin,Sampling Depth) [replicates]
# ignore Nomdepth because it seems to not always be accurate
chl2=pd.DataFrame(chl.groupby(['Date','Station','Niskin','Sampling Depth'],as_index=False).mean())

In [7]:
chl2

,Date,Station,Niskin,Sampling Depth,CTD Cast Rep,Chla_Lab,Chla_QC,Chla_QA,Chla_SampleFieldReplicateNumber
0,2018-01-10,PSS019,9.0,31.196,1.0,0.0710,2.0,3.0,1.0
1,2018-01-10,PSS019,11.0,11.404,1.0,0.1960,2.0,3.0,1.0
2,2018-01-10,PSS019,12.0,1.187,1.0,0.6673,2.0,3.0,1.0
3,2018-01-10,SAR003,9.0,31.057,1.0,0.0530,2.0,3.0,1.0
4,2018-01-10,SAR003,11.0,11.146,1.0,0.2097,2.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...
1787,2019-12-19,NSQ002,9.0,30.676,1.0,0.3154,2.0,2.0,1.0
1788,2019-12-19,NSQ002,10.0,10.871,1.0,0.3640,2.0,2.0,1.0
1789,2019-12-19,NSQ002,12.0,1.214,1.0,0.4397,2.0,2.0,1.0
1790,2019-12-19,OAK004,11.0,11.106,1.0,0.5765,2.0,2.0,2.0


In [8]:
bot2=pd.DataFrame(bot.groupby(['Date','Station','Niskin','Sampling Depth'],as_index=False).mean())

In [9]:
allbot = pd.merge(left=bot2,right=chl2,how='outer',
                    left_on = ['Date','Station','Niskin','CTD Cast Rep','Sampling Depth'],
                    right_on = ['Date','Station','Niskin','CTD Cast Rep','Sampling Depth'])
print(f'len(allbot):{len(allbot)}, len(chl2):{len(chl2)}, len(bot2):{len(bot2)}')

len(allbot):2357, len(chl2):1792, len(bot2):2348


In [10]:
nutNoChl=len(allbot.loc[(~pd.isnull(allbot.NH4_Lab))&(pd.isnull(allbot.Chla_Lab))])
chlNoNut=len(allbot.loc[(pd.isnull(allbot.NH4_Lab))&(~pd.isnull(allbot.Chla_Lab))])
print(nutNoChl,chlNoNut)

565 9


In [11]:
# if these equal the total length, things are making sense:
print(len(bot2)+chlNoNut,len(chl2)+nutNoChl)

2357 2357


In [12]:
allbot.keys()

Index(['Date', 'Station', 'Niskin', 'Sampling Depth', 'CTD Cast Rep',
       'NH4_Lab', 'NH4_QC', 'NH4_QA', 'NH4_SampleFieldReplicateNumber',
       'NO2_Lab', 'NO2_QC', 'NO2_QA', 'NO2_SampleFieldReplicateNumber',
       'NO3_Lab', 'NO3_QC', 'NO3_QA', 'NO3_SampleFieldReplicateNumber',
       'PO4_Lab', 'PO4_QC', 'PO4_QA', 'PO4_SampleFieldReplicateNumber',
       'SiOH4_Lab', 'SiOH4_QC', 'SiOH4_QA', 'SiOH4_SampleFieldReplicateNumber',
       'Unnamed: 36', 'Chla_Lab', 'Chla_QC', 'Chla_QA',
       'Chla_SampleFieldReplicateNumber'],
      dtype='object')

#### Now add date/times

In [13]:
dfTime=pd.read_excel('/ocean/eolson/MEOPAR/obs/WADE/WDE_Data/OlsonSuchyAllen_UBC_PDR_P003790-010721.xlsx',
                    engine='openpyxl',sheet_name='EventDateTime')

In [14]:
## duplicate Station/Date entries with different times seem to be always within a couple of hours, 
# so just take the first (next cell)
test=dfTime.groupby(['FlightDate','SiteCode'])['TimeDown \n(Local - PST or PDT)'].count()
for date, loc in test[test>1].index:
    print(dfTime.loc[(dfTime.FlightDate==date)&(dfTime.SiteCode==loc),['FlightDate','SiteCode','TimeDown \n(Local - PST or PDT)']])

    FlightDate SiteCode TimeDown \n(Local - PST or PDT)
590 2001-07-09   DNA001                        17:09:00
591 2001-07-09   DNA001                        17:09:00
    FlightDate SiteCode TimeDown \n(Local - PST or PDT)
948 2003-06-25   BUD005                        17:00:00
949 2003-06-25   BUD005                        17:00:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
1362 2005-08-16   BUD005                        10:43:00
1363 2005-08-16   BUD005                        10:43:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
1429 2005-11-07   BUD005                        11:55:00
1430 2005-11-07   BUD005                        11:55:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
1446 2005-12-05   BUD005                        11:58:00
1447 2005-12-05   BUD005                        11:58:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
2104 2008-08-21   SJF000                        09:59:00
2105 2008-08-21   SJF000             

In [15]:
# drop duplicate rows
dfTime.drop_duplicates(subset=['FlightDate','SiteCode'],keep='first',inplace=True)
print(dfTime.keys())

Index(['FlightYear', 'FlightMonth', 'FlightDate', 'SiteCode', 'Sampled',
       'TimeDown \n(Local - PST or PDT)', 'FieldComment'],
      dtype='object')


In [16]:
dfTime['dtPac']=[dt.datetime.combine(idate, itime) for idate, itime \
         in zip(dfTime['FlightDate'],dfTime['TimeDown \n(Local - PST or PDT)'])]
dfTime['dtUTC']=[et.pac_to_utc(ii) for ii in dfTime['dtPac']]

In [17]:
allbot['dtUTC']=np.nan # create column and set all values to nan

In [18]:
# use loop to set dtUTC values, where available (where dates are unavailable, left as NaN for now)
for ind, row in allbot.iterrows():
    ix=(dfTime.FlightDate==row['Date'])&(dfTime.SiteCode==row['Station'])
    if np.sum(ix)==1:
        idate,itime=dfTime.loc[ix,['FlightDate','TimeDown \n(Local - PST or PDT)']].values[0]
        allbot.loc[ind,['UTCDateTime']]=et.pac_to_utc(dt.datetime.combine(idate,itime))

#### Now add lat/lons

In [19]:
# PROCESS STATION LOCATION INFO (based on Parker's code)
sta_fn='/ocean/eolson/MEOPAR/obs/WADE/WDE_Data/OlsonSuchyAllen_UBC_PDR_P003790-010721.xlsx'
sheetname='Site Info'
sta_df =pd.read_excel(sta_fn,engine='openpyxl',sheet_name=sheetname)
sta_df.dropna(how='any',subset=['Lat_NAD83 (deg / dec_min)','Long_NAD83 (deg / dec_min)','Station'],inplace=True)
sta_df = sta_df.set_index('Station')
# get locations in decimal degrees
for sta in sta_df.index:
    lat_str = sta_df.loc[sta, 'Lat_NAD83 (deg / dec_min)']
    lat_deg = float(lat_str.split()[0]) + float(lat_str.split()[1])/60
    sta_df.loc[sta,'Lat'] = lat_deg
    #
    lon_str = sta_df.loc[sta, 'Long_NAD83 (deg / dec_min)']
    lon_deg = float(lon_str.split()[0]) + float(lon_str.split()[1])/60
    sta_df.loc[sta,'Lon'] = -lon_deg    
sta_df.pop('Lat_NAD83 (deg / dec_min)');
sta_df.pop('Long_NAD83 (deg / dec_min)');

In [20]:
allbotFinal=pd.merge(left=sta_df,right=allbot,how='right',
                 left_on='Station',right_on='Station')

In [21]:
allbotFinal

,Station,Desig,Descrip,Basin,*Max_Depth,Lat,Lon,Date,Niskin,Sampling Depth,...,SiOH4_QC,SiOH4_QA,SiOH4_SampleFieldReplicateNumber,Unnamed: 36,Chla_Lab,Chla_QC,Chla_QA,Chla_SampleFieldReplicateNumber,dtUTC,UTCDateTime
0,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10,3,26.820,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10,11,11.252,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10,12,1.336,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07,11,10.865,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07,12,1.170,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-06-04,12,1.220,...,2.0,2.0,1.0,NaN,6.2169,2.0,2.0,1.0,NaN,NaN
2353,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-07-02,11,5.963,...,2.0,2.0,1.0,NaN,6.7821,2.0,2.0,1.0,NaN,NaN
2354,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-07-02,12,1.469,...,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2355,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-09-11,11,6.032,...,2.0,2.0,1.0,NaN,9.7210,2.0,2.0,1.0,NaN,NaN


In [22]:
np.unique(allbotFinal.loc[np.isnan(allbotFinal.Lat),['Station']])

array(['BLL040', 'HCB013'], dtype=object)

In [23]:
# There are two station designations that are not pressent in the station list. Drop them.
allbotFinal.dropna(how='any',subset=['Lat','Lon'],inplace=True)

In [24]:
# rename Sample Depth to Z
allbotFinal.rename(columns={'Sampling Depth':'Z'},inplace=True)

In [25]:
allbotFinal

,Station,Desig,Descrip,Basin,*Max_Depth,Lat,Lon,Date,Niskin,Z,...,SiOH4_QC,SiOH4_QA,SiOH4_SampleFieldReplicateNumber,Unnamed: 36,Chla_Lab,Chla_QC,Chla_QA,Chla_SampleFieldReplicateNumber,dtUTC,UTCDateTime
22,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,3,98.380,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10 20:34:00
23,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,9,31.196,...,2.0,3.0,1.0,NaN,0.0710,2.0,3.0,1.0,NaN,2018-01-10 20:34:00
24,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,11,11.404,...,2.0,3.0,1.0,NaN,0.1960,2.0,3.0,1.0,NaN,2018-01-10 20:34:00
25,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,12,1.187,...,2.0,3.0,1.0,NaN,0.6673,2.0,3.0,1.0,NaN,2018-01-10 20:34:00
26,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-03-07,3,86.010,...,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07 21:27:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-06-04,12,1.220,...,2.0,2.0,1.0,NaN,6.2169,2.0,2.0,1.0,NaN,NaN
2353,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-07-02,11,5.963,...,2.0,2.0,1.0,NaN,6.7821,2.0,2.0,1.0,NaN,NaN
2354,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-07-02,12,1.469,...,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2355,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-09-11,11,6.032,...,2.0,2.0,1.0,NaN,9.7210,2.0,2.0,1.0,NaN,NaN


### here is an example loading the new chlorophyll data (includes all years)

In [26]:
fn='/ocean/eolson/MEOPAR/obs/WADE/WDE_Data/OlsonSuchyAllen_UBC_PDR_P003790-010721.xlsx'
sheetname='LabChlaPheo'
chlPheo =pd.read_excel(fn,engine='openpyxl',sheet_name=sheetname)

In [27]:
chlPheo.dropna(how='any',subset=['Date','Station','SamplingDepth'],inplace=True)

In [28]:
# average over replicates
chlPheo2=pd.DataFrame(chlPheo.groupby(['Date','Station','SamplingDepth'],as_index=False).mean())

In [29]:
# join to station info (lat/lon)
chlPheo3=pd.merge(left=sta_df,right=chlPheo2,how='right',
                 left_on='Station',right_on='Station')

In [30]:
# join to date/time
dfTime['dtUTC']=[et.pac_to_utc(dt.datetime.combine(idate,itime)) for idate,itime in \
                zip(dfTime['FlightDate'],dfTime['TimeDown \n(Local - PST or PDT)'])]
dfTime2=dfTime.loc[:,['FlightDate','SiteCode','dtUTC']]
chlPheoFinal=pd.merge(left=chlPheo3,right=dfTime2,how='left',
                      left_on=['Date','Station'],right_on=['FlightDate','SiteCode'])

In [31]:
len(chlPheoFinal),len(chlPheo3),len(dfTime2)

(11469, 11469, 5186)

### Stuff that helped id problems in matching chl and nuts:

In [22]:
def intna(x):
    try: 
        y=int(x)
    except:
        y=np.nan
    return y

In [32]:
np.abs(140-178)/178

0.21348314606741572

In [33]:
nd=[intna(ii) for ii in chl['Nomdepth']]
crit=[np.abs(isd-ind)>max(5,0.3*isd) for ind, isd in zip(nd,chl['Sampling Depth'])]

In [37]:
bot.keys()

Index(['Date', 'Station', 'Niskin', 'Nomdepth', 'Sampling Depth',
       'CTD Cast Rep', 'NH4_Lab', 'NH4_QC', 'NH4_QF', 'NH4_QA', 'NH4_Comment',
       'NH4_SampleFieldReplicateNumber', 'NO2_Lab', 'NO2_QC', 'NO2_QF',
       'NO2_QA', 'NO2_Comment', 'NO2_SampleFieldReplicateNumber', 'NO3_Lab',
       'NO3_QC', 'NO3_QF', 'NO3_QA', 'NO3_Comment',
       'NO3_SampleFieldReplicateNumber', 'PO4_Lab', 'PO4_QC', 'PO4_QF',
       'PO4_QA', 'PO4_Comment', 'PO4_SampleFieldReplicateNumber', 'SiOH4_Lab',
       'SiOH4_QC', 'SiOH4_QF', 'SiOH4_QA', 'SiOH4_Comment',
       'SiOH4_SampleFieldReplicateNumber', 'Unnamed: 36'],
      dtype='object')

In [43]:
for i, row in chl.loc[crit].iterrows():
    print(chl.loc[(chl.Date==row['Date'])&(chl.Station==row.Station),
                  ['Date','Station','Niskin','Nomdepth','Sampling Depth',
                  'CTD Cast Rep','Chla_Lab']])
    print(bot.loc[(bot.Date==row['Date'])&(bot.Station==row.Station),
                  ['Date','Station','Niskin','Nomdepth','Sampling Depth',
                  'CTD Cast Rep','NH4_Lab','NO3_Lab']])
    print('-------\n')

          Date Station  Niskin Nomdepth  Sampling Depth  CTD Cast Rep  \
119 2019-02-26  DNA001    12.0        0           1.136           1.0   
120 2019-02-26  DNA001    10.0       10           9.702           1.0   
121 2019-02-26  DNA001    12.0       30           1.136           1.0   

     Chla_Lab  
119    2.2268  
120    1.9555  
121    1.9216  
          Date Station  Niskin Nomdepth  Sampling Depth  CTD Cast Rep  \
163 2019-02-26  DNA001      12        0           1.136             1   
164 2019-02-26  DNA001      10       10           9.702             1   
165 2019-02-26  DNA001       4       30          29.815             1   

     NH4_Lab  NO3_Lab  
163   0.3302  27.5842  
164   0.3106  27.6943  
165   0.2201  27.8900  
-------

          Date Station  Niskin Nomdepth  Sampling Depth  CTD Cast Rep  \
425 2019-05-08  SJF002    12.0        0           0.713           1.0   
426 2019-05-08  SJF002     4.0      140          79.942           1.0   
427 2019-05-08  SJF002    

In [11]:
nomd=[ii if np.isfloat]

63 nan 10J
66 4.928 10E
68 3.058 10E
76 3.978 10E
155 68.32 NB
179 127.414 NB
206 nan 30J
238 4.313 NB
365 160.141 30E
369 nan 10J
370 nan 10J
371 nan 10J
445 6.025 NB
463 3.254 10E
500 nan 30J
502 0.543 10E
551 nan 10J
667 nan 10E
706 11.04 NB
769 0.921 0J
1174 197.968 NB
1191 nan 10J
1197 nan 10J
1231 143.572 NB
1243 142.451 30E
1462 19.075 30E
1495 89.882 30E
1544 153.77 30E
1554 94.466 30E
1577 1.056 10E
1692 138.64 NB
1694 4.019 10E
1724 2.057 10E
1760 4.133 10E
1944 4.801 NB
1949 5.542 0E
1955 5.517 NB
2113 0.843 10E


In [21]:
allbot = pd.merge(left=bot,right=chl,how='outer',
                    left_on = ['Date','Station','Niskin','CTD Cast Rep','Sampling Depth','NO3_SampleFieldReplicateNumber'],
                    right_on = ['Date','Station','Niskin','CTD Cast Rep','Sampling Depth','Chla_SampleFieldReplicateNumber'])
print(f'len(allbot):{len(allbot)}, len(chl):{len(chl)}, len(bot):{len(bot)}')
#,'Nomdepth'

len(allbot):2760, len(chl):2133, len(bot):2673


In [ ]:
bot.keys()

In [ ]:
chl.keys()

In [ ]:
allbot.keys()

In [ ]:
allbot['ones']=1

In [ ]:
allbot.loc[(~pd.isnull(allbot.NH4_Lab))&(pd.isnull(allbot.Chla_Lab)),['Date','Station','Niskin','CTD Cast Rep','Sampling Depth','NH4_Lab','Chla_Lab']]

In [ ]:
allbot.loc[(pd.isnull(allbot.NH4_Lab))&(~pd.isnull(allbot.Chla_Lab)),['Date','Station','Niskin','CTD Cast Rep','Sampling Depth','NH4_Lab','Chla_Lab']]

In [ ]:
bot[['Date','Station','Niskin','Sampling Depth']].head(5)

In [ ]:
chl[['Date','Station','Niskin','Sampling Depth']].head(5)

In [ ]:
allbot.groupby(['Date','Station','Niskin','Sampling Depth'])['ones'].count()

In [ ]:
temp=allbot.groupby(['Date','Station','Niskin','Sampling Depth']).agg({'ones':['count']})
temp.columns = ['icount']
np.unique(temp.icount)

In [ ]:
temp.loc[temp.icount>1]

In [ ]:
bot.loc[(bot.Date==dt.datetime(2018,1,12))&(bot.Station=='OAK004')]

In [ ]:
chl.loc[(chl.Date==dt.datetime(2018,1,12))&(chl.Station=='OAK004')]

In [ ]:
temp.loc[temp.icount==2]

In [ ]:
bot.loc[(bot.Date==dt.datetime(2019,2,26))&(bot.Station=='DNA001'),['Date','Station','Niskin','Nomdepth','Sampling Depth','NH4_Lab','NO3_Lab']]

In [ ]:
chl.loc[(chl.Date==dt.datetime(2019,2,26))&(chl.Station=='DNA001')]

In [ ]:
allbot.loc[(allbot.Date==dt.datetime(2019,2,26))&(allbot.Station=='DNA001'),['Date','Station','Niskin','Sampling Depth','Chla_Lab']]

In [ ]:
#remove this entry in chl:
    #Date	Station	Niskin	Nomdepth	Sampling Depth	CTD Cast Rep	Chla_Lab	Chla_QC	
    #2019-02-26	DNA001	12.0	30	1.136	1.0	1.9216	2.0	

In [ ]:
allbot.loc[(allbot.Nomdepth_x!=allbot.Nomdepth_y)]